In [1]:
# import findspark
# findspark.init('/home/ductien/yspark-3.3.2-bin-hadoop3')
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import *
import mysql.connector

In [2]:
#Create SparkSession
spark = SparkSession.builder \
                    .config("spark.jars", "./mysql-connector-java-8.0.29.jar")\
                    .appName("MySQL_loader").getOrCreate()

In [3]:
mark_2017_df = spark.read.csv("./CleanedDatasets/Mark2017", header= True, inferSchema= True)

In [4]:
mark_18_22_df = spark.read.csv("./CleanedDatasets/Mark_2018_2022", header= True, inferSchema= True)

<h1> Spark dataframe to MySQL database

In [5]:
import os
from dotenv import load_dotenv
load_dotenv(".env")
dbName = os.getenv("DB_NAME")
dbUser = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")

<h3> Load to province table

In [ ]:
mark_2017_df.createTempView("PROVINCE")

In [ ]:
provinceDF = spark.sql("SELECT DSTINCT province_code, province_name FROM PROVINCE")

In [ ]:
provinceDF.printSchema()

In [ ]:
provinceDF = provinceDF.sort("province_code")

In [ ]:
provinceDF.count()

In [ ]:
from pyspark.sql.functions import col

In [ ]:
newRow = spark.createDataFrame([(39,'phuyen')], provinceDF.columns)
provinceDF = provinceDF.union(newRow)

In [ ]:
newRow = spark.createDataFrame([(45,'ninhthuan')], provinceDF.columns)
provinceDF = provinceDF.union(newRow)

In [ ]:
provinceDF = provinceDF.sort('province_code')

In [ ]:
provinceDF = provinceDF.withColumnRenamed('province', 'ProvinceName') \
                        .withColumnRenamed('province_code', 'provinceCode')

In [ ]:
provinceDF.printSchema()

In [ ]:
provinceDF.count()

In [ ]:
provinceDF.write.format('jdbc').options(
    url = 'jdbc:mysql://localhost:3306/equivalent_score',
    driver = 'com.mysql.jdbc.Driver',
    dbtable = "DimProvince",
    user = dbUser,
    password = password
).mode('append').save()

<h3> Load to Year table

In [ ]:
from pyspark.sql import Row

In [ ]:
yearDF = spark.createDataFrame([
    Row(year = 2018),
    Row(year = 2019),
    Row(year = 2020),
    Row(year = 2021),
    Row(year = 2022)
])

In [ ]:
yearDF.show()

In [ ]:
yearDF.write.format('jdbc').options(
    url = 'jdbc:mysql://localhost:3306/equivalent_score',
    driver = 'com.mysql.jdbc.Driver',
    dbtable = "DimYear",
    user = dbUser,
    password = password
).mode('append').save()

<h2> Load to DimUniverity

In [ ]:
benchmark_df.createTempView('UNI')

In [ ]:
benchmark_df.printSchema()

In [ ]:
uniDF = spark.sql("SELECT uni_code AS UniCode, uni_name AS UniName, \
                   major_code AS MajorCode, major_name AS MajorName, \
                   subject_group AS SubjectGroup, benchmark AS BenchMark, year AS Year FROM UNI")

In [ ]:
from pyspark.sql.functions import count, when, isnan, isnull

In [ ]:
uniDF.select([count(when(isnan(c) | isnull(c), c)).alias(c) for c in uniDF.columns]).show()

In [ ]:
uniDF = uniDF.dropna(subset= uniDF.columns)

In [ ]:
uniDF.select([count(when(isnan(c) | isnull(c), c)).alias(c) for c in uniDF.columns]).show()

In [ ]:
from pyspark.sql.functions import max

In [ ]:
uniDF = uniDF.filter(col('BenchMark') <= 32.75)

In [ ]:
spark.sql('SELECT max(length(major_code)) FROM UNI').show()

In [ ]:
uniDF.count()

In [ ]:
uniDF.write.format('jdbc').options(
    url = 'jdbc:mysql://localhost:3306/equivalent_score',
    driver = 'com.mysql.jdbc.Driver',
    dbtable = "University",
    user = dbUser,
    password = password
).mode('append').save()

<h1> Load to FactScore

In [ ]:
mark_18_22_df.createTempView('student1822')

In [ ]:
mark_18_22_df.printSchema()

In [ ]:
student1822 = spark.sql("SELECT student_id as studentID, literature, mathematics as math, \
                         english, physics, chemistry, biology, history, geography, \
                         civic_education as civil, province_code as ProvinceCode, Year as year FROM student1822")

In [ ]:
student1822 = student1822.distinct()

In [ ]:
student1822.select('ProvinceCode').distinct().count()

In [ ]:
distinctProv = spark.sql("SELECT DISTINCT province_code FROM student1822")

In [ ]:
distinctProv.count()

In [ ]:
student1822.count()

In [ ]:
student1822.select([count(when(isnan(c) | isnull(c), c)).alias(c) for c in student1822.columns]).show()

In [ ]:
student1822.select(max(col('literature'))).show()

In [ ]:
student1822.write.format('jdbc').options(
    url = 'jdbc:mysql://localhost:3306/equivalent_score',
    driver = 'com.mysql.jdbc.Driver',
    dbtable = "FactScore",
    user = dbUser,
    password = password
).partitionBy('year').mode('append').save()